In [9]:
import os
import datetime
import h5py as h5py
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Concatenate, MaxPooling2D, Activation, Flatten, Dropout, Dense,BatchNormalization
from tensorflow.keras.utils import to_categorical,Progbar
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential

In [10]:
print(os.getcwd())

C:\Users\emera


In [11]:
IMG_SHAPE  = (256, 256)
BATCH_SIZE = 32

In [12]:
trainImgGenerator = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    zoom_range = 0.2,
    shear_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest')

valImgGenerator = ImageDataGenerator(
    rescale = 1./255)

In [13]:
maindir = r'F:\emeraldfile\2021-2022S1\Project\project\images_handheld.tar_2'
dataset = pd.read_csv(maindir + "\\annotations_handheld.csv", usecols=[0, 1, 2, 3, 4],
                          names=['image', 'x1', 'y1', 'x2', 'y2'])
img_files = os.listdir(maindir + "\\images_handheld")
dataset['flag'] = dataset['x1'] + dataset['y1'] + dataset['x2'] + dataset['y2'] == 0

imgData = []
labels = []
i = 0
for file in img_files:
    i += 1
    name = dataset.loc[dataset['image'] == file]
    label = name.iloc[0, 5]
    l = 1
    if label :
        l = 0
    path = maindir + "\\images_handheld\\" + file
    img = load_img(path)
    img = img.resize(IMG_SHAPE)
    x = img_to_array(img)
    imgData.append(x)
    labels.append(l)
    if i > 1525:
        break

appendir = r'F:\emeraldfile\2021-2022S1\Project\project\images_handheld.tar_2'
dataset = pd.read_csv(appendir + "\\annotations_test.csv", usecols=[0, 1, 2, 3, 4],
                          names=['image', 'x1', 'y1', 'x2', 'y2'])
img_files = os.listdir(appendir + "\\testset")
dataset['flag'] = dataset['x1'] + dataset['y1'] + dataset['x2'] + dataset['y2'] == 0
i = 0
for file in img_files:
    i += 1
    name = dataset.loc[dataset['image'] == file]
    label = name.iloc[0, 5]
    l = 1
    if label :
        l = 0
    path = appendir + "\\testset\\" + file
    img = load_img(path)
    img = img.resize(IMG_SHAPE)
    x = img_to_array(img)
    imgData.append(x)
    labels.append(l)
    if i > 260:
        break        


In [14]:
h5_train_data = r'F:\emeraldfile\2021-2022S1\Project\project\images_handheld.tar_2\CNN\Output\train_data.h5'
h5_train_labels = r'F:\emeraldfile\2021-2022S1\Project\project\images_handheld.tar_2\CNN\Output\train_labels.h5'

h5fd = h5py.File(h5_train_data, 'w')
h5fd.create_dataset('dataset_1', data=np.array(imgData))

h5fl = h5py.File(h5_train_labels, 'w')
h5fl.create_dataset('dataset_1', data=np.array(labels))

h5fd.close()
h5fl.close()

In [16]:
h5_train_data = r'F:\emeraldfile\2021-2022S1\Project\project\images_handheld.tar_2\CNN\Output\train_data.h5'
h5_train_labels = r'F:\emeraldfile\2021-2022S1\Project\project\images_handheld.tar_2\CNN\Output\train_labels.h5'

h5fd = h5py.File(h5_train_data, 'r')
h5fl = h5py.File(h5_train_labels, 'r')
featuresAll = h5fd['dataset_1']
labelsAll = h5fl['dataset_1']

features = np.array(featuresAll)
labels = np.array(labelsAll)

h5fd.close()
h5fl.close()

In [17]:
x_train, x_test, y_train, y_test = train_test_split(np.array(features), np.array(labels), test_size = 0.3)
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(1250, 256, 256, 3) (1250, 2) (537, 256, 256, 3) (537, 2)


In [18]:
def buildModel():
    model = Sequential()
    
    model.add(MaxPooling2D(pool_size = (7, 1), padding = 'valid'))
    
    model.add(Conv2D(96, (3,3), input_shape = (256, 256, 3)) )
    model.add(Activation('relu'))
    model.add(BatchNormalization(scale = False, center = False))

    model.add(Conv2D(64, (3,3)))
    model.add(BatchNormalization(scale = False, center = False))
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size = (2, 8), padding = 'same'))
    model.add(Dropout(0.2))

    model.add(Conv2D(48, (3,3)))
    model.add(BatchNormalization(scale = False, center = False))
    model.add(Activation('relu'))
    
    model.add(Conv2D(32, (3,3)))
    model.add(BatchNormalization(scale = False, center = False))
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size = (3, 4), padding = 'same'))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    return model

In [19]:
model = buildModel()
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [20]:
epochs=25
for e in range(epochs * 4):
    print('Epoch: ', e)
    print('Start training--------------------------------------------------')
    
    progbar = Progbar(x_train.shape[0])
    batch = 0
    
    for x_batch, y_batch in trainImgGenerator.flow(x_train, y_train, batch_size = BATCH_SIZE, shuffle = True):
        loss,trainingAcc = model.train_on_batch(x_batch, y_batch)
        batch+=x_batch.shape[0]
        
        if batch>x_train.shape[0]:
            break
        progbar.add(x_batch.shape[0], values = [('train loss', loss),('train acc', trainingAcc)])

Epoch:  0
Start training--------------------------------------------------
1250/1250 [==============================] - 61s 49ms/step - train loss: 1.3502 - train acc: 0.5456
Epoch:  1
Start training--------------------------------------------------
1250/1250 [==============================] - 59s 48ms/step - train loss: 0.9887 - train acc: 0.5600
Epoch:  2
Start training--------------------------------------------------
1250/1250 [==============================] - 60s 48ms/step - train loss: 0.8941 - train acc: 0.5792
Epoch:  3
Start training--------------------------------------------------
1250/1250 [==============================] - 60s 48ms/step - train loss: 0.8083 - train acc: 0.6040
Epoch:  4
Start training--------------------------------------------------
1250/1250 [==============================] - 60s 48ms/step - train loss: 0.7812 - train acc: 0.6024
Epoch:  5
Start training--------------------------------------------------
1250/1250 [==============================] - 60s 4

1250/1250 [==============================] - 61s 49ms/step - train loss: 0.4509 - train acc: 0.7976
Epoch:  47
Start training--------------------------------------------------
1250/1250 [==============================] - 60s 48ms/step - train loss: 0.4564 - train acc: 0.7848
Epoch:  48
Start training--------------------------------------------------
1250/1250 [==============================] - 60s 48ms/step - train loss: 0.4541 - train acc: 0.7992
Epoch:  49
Start training--------------------------------------------------
1250/1250 [==============================] - 61s 48ms/step - train loss: 0.4536 - train acc: 0.7976
Epoch:  50
Start training--------------------------------------------------
1250/1250 [==============================] - 65s 52ms/step - train loss: 0.4752 - train acc: 0.7712
Epoch:  51
Start training--------------------------------------------------
1250/1250 [==============================] - 60s 48ms/step - train loss: 0.4611 - train acc: 0.7832
Epoch:  52
Start tra

Epoch:  93
Start training--------------------------------------------------
1250/1250 [==============================] - 60s 48ms/step - train loss: 0.3973 - train acc: 0.8144
Epoch:  94
Start training--------------------------------------------------
1250/1250 [==============================] - 60s 48ms/step - train loss: 0.4048 - train acc: 0.8088
Epoch:  95
Start training--------------------------------------------------
1250/1250 [==============================] - 60s 48ms/step - train loss: 0.3718 - train acc: 0.8304
Epoch:  96
Start training--------------------------------------------------
1250/1250 [==============================] - 60s 48ms/step - train loss: 0.3808 - train acc: 0.8320
Epoch:  97
Start training--------------------------------------------------
1250/1250 [==============================] - 60s 48ms/step - train loss: 0.3825 - train acc: 0.8288
Epoch:  98
Start training--------------------------------------------------
1250/1250 [==============================] -

In [22]:
vdata = valImgGenerator.flow(x_test,y_test,
    batch_size = BATCH_SIZE,
    shuffle = True)
loss, accuracy = model.evaluate(vdata)
print('Loss: ', loss)
print('Accuracy: ', accuracy)

17/17 [==============================] - 3s 165ms/step - loss: 0.3696 - accuracy: 0.8454
Loss:  0.3696210980415344
Accuracy:  0.8454376459121704
